In [ ]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
mv_prefix = 'mv_'

max_workers = 1
max_editors = 100
email = 'd.ecer@elifesciences.org'

In [ ]:
import logging
from datetime import datetime
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import pandas as pd
from tqdm.auto import tqdm

import data_science_pipeline.configure_warnings  # pylint: disable=unused-import
import data_science_pipeline.configure_notebook_logging  # pylint: disable=unused-import

from data_science_pipeline.utils.requests import requests_retry_session
from data_science_pipeline.utils.pubmed import PubmedBibliographyScraper
from data_science_pipeline.utils.bq import to_gbq, is_bq_not_found_exception
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query,
)

In [ ]:
LOGGER = logging.getLogger(__name__)

logging.basicConfig(level='INFO')
logging.getLogger('data_science_pipeline').setLevel(logging.INFO)

In [ ]:
editor_parsed_pubmed_links_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_links'
)

editor_parsed_pubmed_ids_table_name = '{output_dataset}.{mv_prefix}{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    mv_prefix=mv_prefix,
    prefix=output_table_prefix,
    suffix='editor_pubmed_ids'
)

In [ ]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [ ]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [ ]:
_sql = (
    'SELECT person_id FROM `{table}`'
).format(table=editor_parsed_pubmed_ids_table_name)

try:
    existing_editor_ids_df = read_big_query(_sql)
except Exception as e:  # pylint: disable=broad-except
    if not is_bq_not_found_exception(e):
        raise
    print('table not found: %s' % editor_parsed_pubmed_ids_table_name)
    existing_editor_ids_df = pd.DataFrame(columns=['person_id'])
print(len(existing_editor_ids_df))
existing_editor_ids_df.head()

In [ ]:
existing_editor_ids_set = set(existing_editor_ids_df['person_id'])
len(existing_editor_ids_set)

In [ ]:
editor_parsed_pubmed_links_df = read_big_query(
    'SELECT * FROM `{table_name}`\nWHERE is_ncbi_bibliography_url'.format(
        table_name=editor_parsed_pubmed_links_table_name
    )
)
print(len(editor_parsed_pubmed_links_df))
editor_parsed_pubmed_links_df.head()

In [ ]:
remaining_editor_parsed_pubmed_links_df = editor_parsed_pubmed_links_df[
    ~editor_parsed_pubmed_links_df['person_id'].isin(existing_editor_ids_set)
]
print(len(remaining_editor_parsed_pubmed_links_df))
remaining_editor_parsed_pubmed_links_df.head(3)

In [ ]:
processing_editor_parsed_pubmed_links_df = remaining_editor_parsed_pubmed_links_df
if max_editors:
    processing_editor_parsed_pubmed_links_df = processing_editor_parsed_pubmed_links_df[:max_editors]
len(processing_editor_parsed_pubmed_links_df)

In [ ]:
editor_pubmed_links_result_df = processing_editor_parsed_pubmed_links_df[:max_editors].copy()

with requests_retry_session() as session:
    scraper = PubmedBibliographyScraper(session)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        editor_pubmed_links_result_df['pubmed_ids'] = list(tqdm(
            executor.map(
                scraper.get_pmids,
                editor_pubmed_links_result_df['resolved_pubmed_url']
            ),
            total=len(editor_pubmed_links_result_df),
            leave=False
        ))

In [ ]:
non_empty_editor_pubmed_links_result_df = editor_pubmed_links_result_df[
    ~pd.isnull(editor_pubmed_links_result_df['pubmed_ids'])
].copy()
len(non_empty_editor_pubmed_links_result_df)

In [ ]:
non_empty_editor_pubmed_links_result_df['provenance'] = [{
    'source': 'europepmc',
    'imported_timestamp': datetime.utcnow().isoformat()
}] * len(non_empty_editor_pubmed_links_result_df)

In [ ]:
if len(non_empty_editor_pubmed_links_result_df) == 0:
    print('no data to upload')
else:
    print('writing to:', editor_parsed_pubmed_ids_table_name)
    to_gbq(
        df=non_empty_editor_pubmed_links_result_df,
        project_id=project_id,
        destination_table=editor_parsed_pubmed_ids_table_name,
        if_exists='append'
    )
    print('done')